In [1]:
from datetime import date, datetime

import pandas as pd
import pytz
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

local_tz = pytz.timezone("Asia/Dubai")

In [2]:
date_fr = date.today().strftime("%m/%d/%Y %H:%M:%S")
date_to = datetime.now(local_tz).strftime("%m/%d/%Y %H:%M:%S")

In [3]:
date_fr, date_to

('04/19/2024 00:00:00', '04/19/2024 07:33:23')

In [4]:
db = Database()

reservation = db.gwg_reservation

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [5]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

In [6]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 3).get()

    df = pipeline.fit_transform(booking_data)
    dfs.append(df)

In [7]:
records = pd.concat(dfs, axis=0, ignore_index=True)

if records is not None and not records.empty:
    chunk_size = 1000
    for i in range(0, len(records), chunk_size):
        chunk = records.iloc[i : i + chunk_size]

        stmt = insert(reservation).values(chunk.to_dict(orient="records"))
        stmt = stmt.on_conflict_do_update(
            constraint="gwg_reservation_pkey", set_=dict(stmt.excluded)
        )

        with db.engine.begin() as connection:
            connection.execute(stmt)